In [ ]:
from chunking import extract_text_from_pdf

pages = extract_text_from_pdf(start_page=1)

page_docs=[]
for page_num, content in pages.items():
    text_per_page = content.get("text")
    page_docs.append({
        "page_num": page_num,
        "text": text_per_page
    })
page_docs


In [4]:
# results = []
# for page_num, payload in pages.items():
#     page = {
#         "page": page_num,
#         "chapter": payload["chapter"],
#         "text": payload["text"],
#     }
#     result = process_document(chunk)
#     result["page"] = page_num
#     results.append(result)

In [2]:
from openai import OpenAI
openai_client = OpenAI()

def llm_structured(instructions, user_prompt, output_format, model="gpt-4o-mini"):
    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_prompt}
    ]

    response = openai_client.responses.parse(
        model=model,
        input=messages,
        text_format=output_format
    )

    return (response.output_parsed, response.usage)

In [3]:
from pydantic import BaseModel, Field
from typing import List, Literal

class Question(BaseModel):
    """
    Represents a realistic search-engine-style query a user might type before finding the article.
    """

    question: str = Field(
        ...,
        description="A natural, short search query — not a full-sentence question — phrased like something typed into Google.",
    )
    summary_answer: str = Field(
        ...,
        description="A concise 1–2 sentence summary of how the article addresses the query.",
    )
    difficulty: Literal["beginner", "intermediate", "expert"] = Field(
        ..., description="The assumed knowledge level of the ai engineer making the query."
    )



class GeneratedQuestions(BaseModel):
    """
    A structured collection of human-like search queries derived from a given article.
    """

    description: str = Field(
        ...,
        description="A summary of the article or topic these search-style questions were generated for.",
    )
    questions: List[Question] = Field(
        ...,
        description="A list of realistic search queries with short summaries, difficulty levels, and user intent.",
    )

instructions = """ 
    You are given a page from a chapter called Evaluating AI Systems from a technical AI Engineering textbook.
    Your task is to imagine what an AI engineer might type into a search engine to learn about evaluating LLMs. 

    Generate realistic, human-like search queries — not formal questions. 

    Guidelines:
  
    - Make queries varied and spontaneous, not repetitive or over-polished.
    - Assume users of different knowledge levels from beginner (think junior level) to expert (think ai researcher or senior level):
        - beginner: broad or basic understanding
        - intermediate: knows basic terms but seeks clarification 
        - expert: familiar with evaluating 

    Distribution rules:
    - 50% of the queries should target beginner-level ai engineers
    - 30% should target intermediate-level ai engineers
    - 20% should target advanced-level ai engineers
 

    For each generated query, include:
    - question: the natural, human-style search phrase
    - summary_answer: a short 1–2 sentence summary of how the chapter chunks addresses it
    - difficulty: one of ["beginner", "intermediate", "expert"]

    Also include a description summarizing what kind of article the questions are about.
    """.strip()

In [4]:
from tqdm import tqdm

def map_progress(pool, seq, f):
    """Map function f over seq using the provided executor pool while
    displaying a tqdm progress bar. Returns a list of results in submission order.
    """
    results = []
    
    with tqdm(total=len(seq)) as progress:
        futures = []
    
        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)
        
        return results

In [5]:
import json

def process_document(doc:dict):
    content = doc.get("text", "")

    num_questions = len(content) // 400 # create one question for every 400 characters

    user_prompt = f"""
    generate {num_questions} questions for this document:
    {json.dumps(doc)}
    """.strip()

    output, usage = llm_structured(
        instructions=instructions,
        user_prompt=user_prompt,
        output_format=GeneratedQuestions,
    )

    final_output = {'page': doc, 'questions': output, 'usage': usage}
    return final_output

In [6]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=6) as pool:
    results = map_progress(pool, page_docs, process_document)

results

100%|██████████| 100/100 [01:20<00:00,  1.24it/s]


[{'page': {'page_num': 1,
   'text': 'Chapter 4. Evaluate AI Systems\nA model is only useful if it works for its intended purposes. You need to\nevaluate models in the context of your application. Chapter 3 discusses\ndifferent approaches to automatic evaluation. This chapter discusses how to\nuse these approaches to evaluate models for your applications.\nThis chapter contains three parts. It starts with a discussion of the criteria\nyou might use to evaluate your applications and how these criteria are\ndefined and calculated. For example, many people worry about AI making\nup facts—how is factual consistency detected? How are domain-specific\ncapabilities like math, science, reasoning, and summarization measured?\nThe second part focuses on model selection. Given an increasing number of\nfoundation models to choose from, it can feel overwhelming to choose the\nright model for your application. Thousands of benchmarks have been\nintroduced to evaluate these models along different cri

In [7]:
from toyaikit.pricing import PricingConfig

pricing = PricingConfig()
input_tokens = 0
output_tokens = 0

for r in results:
    usage = r['usage']
    input_tokens = input_tokens + usage.input_tokens
    output_tokens = output_tokens + usage.output_tokens
    
pricing.calculate_cost('gpt-4o-mini', input_tokens, output_tokens)

CostInfo(input_cost=Decimal('0.01202535'), output_cost=Decimal('0.01017'), total_cost=Decimal('0.02219535'))

In [10]:
final_questions = []

for r in results:
    page = r.get("page")
    page_num = page.get("page_num")
    doc_text = page.get("text", "")
    questions = r.get("questions")
    for q in questions.questions:
        entry = q.model_dump()
        entry["text"] = doc_text
        final_questions.append(entry)
final_questions

[{'question': 'how to evaluate ai model performance',
  'summary_answer': 'The chapter outlines essential criteria for evaluating AI models, including how to measure factual consistency and domain-specific capabilities relevant to the application.',
  'difficulty': 'beginner',
  'text': 'Chapter 4. Evaluate AI Systems\nA model is only useful if it works for its intended purposes. You need to\nevaluate models in the context of your application. Chapter 3 discusses\ndifferent approaches to automatic evaluation. This chapter discusses how to\nuse these approaches to evaluate models for your applications.\nThis chapter contains three parts. It starts with a discussion of the criteria\nyou might use to evaluate your applications and how these criteria are\ndefined and calculated. For example, many people worry about AI making\nup facts—how is factual consistency detected? How are domain-specific\ncapabilities like math, science, reasoning, and summarization measured?\nThe second part focuse

In [11]:
len(final_questions)

252

In [12]:
import pandas as pd

pd.set_option("display.max_colwidth",None)
df = pd.DataFrame(final_questions)
df


,question,summary_answer,difficulty,text
0,how to evaluate ai model performance,"The chapter outlines essential criteria for evaluating AI models, including how to measure factual consistency and domain-specific capabilities relevant to the application.",beginner,"Chapter 4. Evaluate AI Systems\nA model is only useful if it works for its intended purposes. You need to\nevaluate models in the context of your application. Chapter 3 discusses\ndifferent approaches to automatic evaluation. This chapter discusses how to\nuse these approaches to evaluate models for your applications.\nThis chapter contains three parts. It starts with a discussion of the criteria\nyou might use to evaluate your applications and how these criteria are\ndefined and calculated. For example, many people worry about AI making\nup facts—how is factual consistency detected? How are domain-specific\ncapabilities like math, science, reasoning, and summarization measured?\nThe second part focuses on model selection. Given an increasing number of\nfoundation models to choose from, it can feel overwhelming to choose the\nright model for your application. Thousands of benchmarks have been\nintroduced to evaluate these models along different criteria. Can these\nbenchmarks be trusted? How do you select what benchmarks to use? How\nabout public leaderboards that aggregate multiple benchmarks?\nThe model landscape is teeming with proprietary models and open source\nmodels. A question many teams will need to visit over and over again is\nwhether to host their own models or to use a model API. This question has\nbecome more nuanced with the introduction of model API services built on\ntop of open source models."
1,criteria for model evaluation in ai,"It discusses various criteria used to evaluate AI applications, detailing how these criteria can be defined and calculated to ensure model effectiveness.",beginner,"Chapter 4. Evaluate AI Systems\nA model is only useful if it works for its intended purposes. You need to\nevaluate models in the context of your application. Chapter 3 discusses\ndifferent approaches to automatic evaluation. This chapter discusses how to\nuse these approaches to evaluate models for your applications.\nThis chapter contains three parts. It starts with a discussion of the criteria\nyou might use to evaluate your applications and how these criteria are\ndefined and calculated. For example, many people worry about AI making\nup facts—how is factual consistency detected? How are domain-specific\ncapabilities like math, science, reasoning, and summarization measured?\nThe second part focuses on model selection. Given an increasing number of\nfoundation models to choose from, it can feel overwhelming to choose the\nright model for your application. Thousands of benchmarks have been\nintroduced to evaluate these models along different criteria. Can these\nbenchmarks be trusted? How do you select what benchmarks to use? How\nabout public leaderboards that aggregate multiple benchmarks?\nThe model landscape is teeming with proprietary models and open source\nmodels. A question many teams will need to visit over and over again is\nwhether to host their own models or to use a model API. This question has\nbecome more nuanced with the introduction of model API services built on\ntop of open source models."
2,how to choose the right ai model from benchmarks,"The chapter explains the challenges of selecting an appropriate model given numerous benchmarks, including how to determine trustworthy benchmarks and the role of leaderboards in model selection.",intermediate,"Chapter 4. Evaluate AI Systems\nA model is only useful if it works for its intended purposes. You need to\nevaluate models in the context of your application. Chapter 3 discusses\ndifferent approaches to automatic evaluation. This chapter discusses how to\nuse these approaches to evaluate models for your applications.\nThis chapter contains three parts. It starts with a discussion of the criteri

In [ ]:
import pandas as pd

gt = pd.read_csv("generated_questions_chap4.csv")
gt_dict = gt.to_dict(orient="records")

In [15]:
gt_dict[2]

{'question': 'how to choose the right ai model from benchmarks',
 'summary_answer': 'The chapter explains the challenges of selecting an appropriate model given numerous benchmarks, including how to determine trustworthy benchmarks and the role of leaderboards in model selection.',
 'difficulty': 'intermediate',
 'text': 'Chapter 4. Evaluate AI Systems\nA model is only useful if it works for its intended purposes. You need to\nevaluate models in the context of your application. Chapter 3 discusses\ndifferent approaches to automatic evaluation. This chapter discusses how to\nuse these approaches to evaluate models for your applications.\nThis chapter contains three parts. It starts with a discussion of the criteria\nyou might use to evaluate your applications and how these criteria are\ndefined and calculated. For example, many people worry about AI making\nup facts—how is factual consistency detected? How are domain-specific\ncapabilities like math, science, reasoning, and summarizatio

In [20]:
import hashlib
import pandas as pd

def generate_chunk_id(chunk):
    combined = f"{chunk["question"]}-{chunk["text"][:10]}"
    hash_hex = hashlib.md5(combined.encode()).hexdigest()
    chunk_id = hash_hex[:8]
    return chunk_id

for chunk in gt_dict:
    chunk["id"] = generate_chunk_id(chunk)

gt_with_id = pd.DataFrame(gt_dict)

gt_with_id.to_csv("generated_questions_chap4.csv", index=False)



In [1]:
import pandas as pd
pd.reset_option("display.max_colwidth")
pd.read_json("eval_chunk_300_250.json")

,question,input_tokens,output_tokens,total_tokens,input_cost,output_cost,total_cost,answer_relevant,completeness,grounded_accuracy,context_utilization,chunk_coverage,consistency,focused,uncertainty_handling
0,importance of reliable benchmarks in AI,1579,2927,4506,0.000079,0.001171,0.001250,True,True,True,True,True,True,True,True
1,what is evaluation-driven development in ai?,1410,2699,4109,0.000071,0.001080,0.001150,True,True,True,True,True,True,True,True
2,performance differences in model APIs,1587,2418,4005,0.000079,0.000967,0.001047,True,True,True,True,True,True,True,True
3,why do OpenAI models seem worse after updates,1597,2574,4171,0.000080,0.001030,0.001109,True,True,True,True,True,True,True,True
4,Why did Samsung ban ChatGPT?,1402,2452,3854,0.000070,0.000981,0.001051,True,True,True,True,True,True,True,True
5,methods for decontaminating evaluation data,1444,1832,3276,0.000072,0.000733,0.000805,True,True,True,True,False,True,True,False
6,best practices for evaluating generative AI re...,1545,1835,3380,0.000077,0.000734,0.000811,True,True,True,True,True,True,True,True
7,limitations of public benchmarks in AI evaluation,1493,2260,3753,0.000075,0.000904,0.000979,True,False,True,True,True,True,True,False
8,evaluating generative AI models,1593,2850,4443,0.000080,0.001140,0.001220,True,True,True,True,True,True,True,True
9,measuring safety and factual consistency in ge...,1643,2798,4441,0.000082,0.001119,0.001201,True,True,True,True,True,True,True,False
